In [1]:
import torch
from torch import nn
import torch.nn.functional as F 
from torch import optim 
from torch.autograd import Variable
from torchvision import datasets, transforms, models
from PIL import Image
import numpy as np
import pandas as pd
import os 
from matplotlib import pyplot as plt
#from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader

In [16]:
train_transforms = transforms.Compose([transforms.Resize((256)),
                                       transforms.RandomRotation(degrees=15),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225]) 
                                    ])
validation_transforms = transforms.Compose([transforms.Resize((256)),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406],
                                                                 [0.229, 0.224, 0.225])])

In [2]:
test_transforms = transforms.Compose([transforms.Resize((256)), 
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

In [34]:
test_data = datasets.ImageFolder("C:/Users/user/Desktop/fer2013_master/test", transform=test_transforms)
test_loader = DataLoader(test_data, batch_size=32, num_workers=0)

In [3]:
#for RAF-DB
raf_data = datasets.ImageFolder("C:/Users/user/Desktop/fer2013_master/validation", transform=test_transforms)
raf_loader = DataLoader(raf_data, batch_size=32, num_workers=0)

In [17]:
train_data = datasets.ImageFolder("C:/Users/user/Desktop/fer2013_master/Dataset/train", transform=train_transforms)
validation_data = datasets.ImageFolder("C:/Users/user/Desktop/fer2013_master/Dataset/test", transform=validation_transforms)
num_workers = 0

train_loader = DataLoader(train_data, batch_size=32, num_workers=num_workers)
validation_loader = DataLoader(validation_data, batch_size=32, num_workers=num_workers)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.densenet121(pretrained = True)
model

C:\Users\user\AppData\Roaming\Python\Python38\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\user\AppData\Roaming\Python\Python38\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [5]:
for param in model.parameters():
    param.requires_grad = False
fc = nn.Sequential(nn.Linear(1024, 64),
                   nn.ReLU(),
                   nn.Dropout(0.3),
                   nn.Linear(64, 7),
                   nn.Softmax(dim=1))
model.classifier = fc
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= 0.01)
model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [21]:
import time
epochs = 10
valid_loss_min = np.Inf
for epoch in range(epochs):

    start = time.time()

    # scheduler.step()
    model.train()

    train_loss = 0.0
    valid_loss = 0.0

    for inputs, labels in train_loader:

        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        logps = model(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    model.eval()

    with torch.no_grad():
        accuracy = 0
        for inputs, labels in validation_loader:

            inputs, labels = inputs.to(device), labels.to(device)
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)
            valid_loss += batch_loss.item()
            # Calculate accuracy
            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()


    # calculate average losses
    train_loss = train_loss/len(train_loader)
    valid_loss = valid_loss/len(validation_loader)
    valid_accuracy = accuracy/len(validation_loader) 
      
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tValidation Accuracy: {:.6f}'.format(
        epoch + 1, train_loss, valid_loss, valid_accuracy))
            
    
    
    if valid_loss <= valid_loss_min:
      
      print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
      valid_loss_min,
      valid_loss))
      model_save_name = "fer2013Inception.pt"
      path = F"C:/Users/user/Desktop/fer2013_master/model{model_save_name}"
      torch.save(model.state_dict(), path)
      valid_loss_min = valid_loss          
      print(f"Time per epoch: {(time.time() - start):.3f} seconds")

Epoch: 1 	Training Loss: 1.864266 	Validation Loss: 1.913323 	Validation Accuracy: 0.246389
Validation loss decreased (inf --> 1.913323).  Saving model ...
Time per epoch: 718.610 seconds
Epoch: 2 	Training Loss: 1.854619 	Validation Loss: 1.918059 	Validation Accuracy: 0.246389
Epoch: 3 	Training Loss: 1.853853 	Validation Loss: 1.918665 	Validation Accuracy: 0.246389
Epoch: 4 	Training Loss: 1.864527 	Validation Loss: 1.918817 	Validation Accuracy: 0.246389
Epoch: 5 	Training Loss: 1.878917 	Validation Loss: 1.918830 	Validation Accuracy: 0.246389
Epoch: 6 	Training Loss: 1.933377 	Validation Loss: 1.908014 	Validation Accuracy: 0.246389
Validation loss decreased (1.913323 --> 1.908014).  Saving model ...
Time per epoch: 704.749 seconds
Epoch: 7 	Training Loss: 1.841784 	Validation Loss: 1.918607 	Validation Accuracy: 0.246389
Epoch: 8 	Training Loss: 1.881044 	Validation Loss: 1.918597 	Validation Accuracy: 0.246389
Epoch: 9 	Training Loss: 1.893537 	Validation Loss: 1.918354 	Valid

In [6]:
model.load_state_dict(torch.load("C:/Users/user/Desktop/fer2013_master/modelfer2013Inception.pt"))

<All keys matched successfully>

In [7]:
def test(model, criterion, test_loader):
    test_loss = 0.
    correct = 0.
    total = 0.
    for batch_idx, (data, target) in enumerate(test_loader):
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
        output = model(data)
        loss = criterion(output, target)
        test_loss = test_loss + ((1 / (batch_idx + 1))*(loss.data - test_loss))
        pred = output.data.max(1, keepdim=True)[1]
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
    print('Test loss: {:.6f}\n'.format(test_loss))
    print('\nTest Accuracy: %2d%%  (%2d/%2d)' % (100. * correct / total, correct, total))


In [38]:
test(model, criterion, test_loader)

Test loss: 1.921332


Test Accuracy: 25%  (1827/7048)


In [8]:
test(model, criterion, raf_loader)

Test loss: 1.900401


Test Accuracy: 25%  (1821/7066)


In [40]:
def load_input_image(img_path):
    image = Image.open(img_path)
    prediction_transform = transforms.Compose([transforms.Resize(size=(224, 224)),
                                               transforms.ToTensor(),
                                               transforms.Normalize([0.485, 0.456, 0.406],
                                                                    [0.229, 0.224, 0.225])])

    # discard the transparent, alpha channel (that's the :3) and add the batch dimension
    image = prediction_transform(image)[:3, :, :].unsqueeze(0)
    return image

In [41]:
def predict_emotion(model, class_names, img_path):
    # load the image and return the predicted breed
    img = load_input_image(img_path)
    model = model.cpu()
    model.eval()
    idx = torch.argmax(model(img))
    return class_names[idx]


In [42]:
class_names = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
img_path = "C:/Users/user/Pictures/Camera Roll/image.jpg"
print(predict_emotion(model, class_names, img_path))

happy


In [14]:
print(torch.cuda.is_available())

True
